In [97]:
import openai
import langchain
import pinecone 
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_pinecone import PineconeVectorStore
from langchain.chat_models import ChatOpenAI
from langchain.embeddings import HuggingFaceEmbeddings

from dotenv import load_dotenv
load_dotenv()
import os


In [28]:
#lest read the documents
def read_doc(directory):
    file_loader=PyPDFDirectoryLoader(directory)
    documents=file_loader.load()
    return documents

In [78]:
doc=read_doc("docs/")
len(doc)

1

In [79]:
#div the docs into chunks
def chunk_data(docs,chunk_size=800,chunk_overlap=50):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=chunk_size,chunk_overlap=chunk_overlap)
    docs=text_splitter.split_documents(docs)
    return docs

In [81]:
documents=chunk_data(docs=doc)
documents

[Document(metadata={'producer': 'Skia/PDF m136', 'creator': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/136.0.0.0 Safari/537.36', 'creationdate': '2025-06-27T07:36:45+00:00', 'title': 'New_Resume', 'moddate': '2025-06-27T07:36:45+00:00', 'source': 'docs/Resume.pdf', 'total_pages': 1, 'page': 0, 'page_label': '1'}, page_content='Kunal Vichave\nComputer Engineer\n (+91) 7387961727  kunalvichave258@gmail.com  kvichave  LinkedIn 17/10/2003\nSkills\nProgramming Languages: Python, JavaScript, HTML, CSS\nFrameworks & Libraries: Flask, Django, React, Next.js, Tailwind CSS\nAI/ML & GenAI: FineTuning, HuggingFace, LangChain, LLMs, RAG, Pandas, NumPy\nWeb Technologies: Jinja, AJAX, Jsx\nData Visualization: Plotly, Chart.js, Matplotlib, Seaborn\nDatabases: MySQL, SQLite3, MongoDB\nTools & Platforms: Git, Docker (learning), Selenium, BeautifulSoup, Heroku, Render, AWS\nEducation\nLokmanya Tilak College of Engineering\n\xa0\xa0\xa0\xa0\xa0- B.E in CSE (AIML), Kopar

NameError: name 'docs' is not defined

In [32]:
embeddings=HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

In [33]:
vectors=embeddings.embed_query("how are you")

In [34]:
len(vectors)

384

In [48]:
os.environ["PINECONE_API_KEY"]=os.getenv("PINECONE_API")

In [66]:
pc=pinecone.Pinecone(
    api_key=os.getenv("PINECONE_API"),
    environment="us-east-1"
)
index_name="pdfchat"


In [84]:
index=PineconeVectorStore.from_documents(documents,embeddings,index_name=index_name)

In [ ]:
# Cosine Similarity: retrive results from VectorDB

def retrive_query(query,k=2):
    matching_results=index.similarity_search(query,k=k)
    return matching_results

In [74]:
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI

In [77]:
doc

[Document(metadata={'producer': 'Skia/PDF m136', 'creator': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/136.0.0.0 Safari/537.36', 'creationdate': '2025-06-27T07:36:45+00:00', 'title': 'New_Resume', 'moddate': '2025-06-27T07:36:45+00:00', 'source': 'docs/Resume.pdf', 'total_pages': 1, 'page': 0, 'page_label': '1', 'text': 'Kunal Vichave\nComputer Engineer\n (+91) 7387961727  kunalvichave258@gmail.com  kvichave  LinkedIn 17/10/2003\nSkills\nProgramming Languages: Python, JavaScript, HTML, CSS\nFrameworks & Libraries: Flask, Django, React, Next.js, Tailwind CSS\nAI/ML & GenAI: FineTuning, HuggingFace, LangChain, LLMs, RAG, Pandas, NumPy\nWeb Technologies: Jinja, AJAX, Jsx\nData Visualization: Plotly, Chart.js, Matplotlib, Seaborn\nDatabases: MySQL, SQLite3, MongoDB\nTools & Platforms: Git, Docker (learning), Selenium, BeautifulSoup, Heroku, Render, AWS\nEducation\nLokmanya Tilak College of Engineering\n\xa0\xa0\xa0\xa0\xa0- B.E in CSE (AIML), Kopar Khair

In [98]:
llm=ChatOpenAI(api_key=os.getenv("OPENAI_API_KEY"),base_url=os.getenv("OPENAI_API_BASE"),
     model_name="sarvamai/sarvam-m:free",
    temperature=0.7)
chain=load_qa_chain(llm,chain_type="stuff")

/tmp/ipykernel_15776/2501683582.py:1: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm=ChatOpenAI(api_key=os.getenv("OPENAI_API_KEY"),base_url=os.getenv("OPENAI_API_BASE"),


In [99]:
llm.predict("What is the capital of France?")

/tmp/ipykernel_15776/2891788140.py:1: LangChainDeprecationWarning: The method `BaseChatModel.predict` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  llm.predict("What is the capital of France?")


'Okay, the user is asking, "What is the capital of France?" Let me start by recalling basic geography. France is a European country, and I remember that Paris is commonly referred to as its capital. But wait, I should double-check to be sure. Sometimes people confuse capitals with other major cities. For example, some countries have capitals that aren\'t the largest cities. But in France\'s case, Paris is both the capital and the largest city. Let me think if there\'s any chance it\'s another city. Maybe Lyon or Marseille? No, I\'m pretty confident it\'s Paris. To confirm, I can recall historical facts. The Eiffel Tower is in Paris, and major events like the Paris Olympics are held there. Also, government buildings like the Louvre and the Palais de Versailles are in Paris. Yeah, definitely Paris. I don\'t think there\'s been a change recently. So the answer should be Paris.\n</think>\n\nThe capital of France is **Paris**.'

In [100]:
# Search answers from VectorDB

def retrive_answer(query):
    doc_search=retrive_query(query)
    print(doc_search)
    response=chain.run(input_documents=doc_search,question=query)
    return response


In [101]:
prompt="what is my name"
answer=retrive_answer(prompt)
print(answer)

[Document(id='d6f14ad0-c4e4-4854-8d17-db0efe8d36cb', metadata={'creationdate': '2025-06-27T07:36:45+00:00', 'creator': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/136.0.0.0 Safari/537.36', 'moddate': '2025-06-27T07:36:45+00:00', 'page': 0.0, 'page_label': '1', 'producer': 'Skia/PDF m136', 'source': 'docs/Resume.pdf', 'title': 'New_Resume', 'total_pages': 1.0}, page_content='02/2024 – 05/2024\nDocBerg - GitHub\n\xa0\xa0\xa0\xa0\xa0- Created a resume analyzer, image-to-text extractor, and document converter\n\xa0\xa0\xa0\xa0\xa0- Built using Flask, Tailwind CSS, JavaScript, TensorFlow, and OCR\n07/2023 – 11/2023\nCogtnitiveBerg - GitHub\n\xa0\xa0\xa0\xa0\xa0- AI interview and business meeting simulator using Llama API and Chart.js\n\xa0\xa0\xa0\xa0\xa0- Built in Flask + React + Next.js for frontend and backend integration\n07/2024 – 04/2025\nMedLLaMA Finetune - GitHub\n\xa0\xa0\xa0\xa0\xa0- Fine-tuned LLaMA on a medical dataset for domain-speciﬁc questi